In [1]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm
import xgboost
import sklearn
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight

/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df_X = pd.read_csv("data/X_train_features_final.csv", index_col=0)
df_Y = pd.read_csv("data/y_train.csv", index_col="id")

In [3]:
from sklearn.model_selection import RandomizedSearchCV

lr = [0.05, 0.1, 0.2]
n_estimators = [100, 200, 400, 600, 800, 1000]
#num_leaves = [11, 31, 51, 101, 201]
#min_data_in_leaf = [11, 21, 31, 51, 101]
max_depth = [-1, 5, 6, 7, 8, 9]


random_grid = {'learning_rate': lr,
               'n_estimators': n_estimators, #'num_leaves': num_leaves,
               'max_depth': max_depth}
               #'min_data_in_leaf': min_data_in_leaf}
print(random_grid)

{'learning_rate': [0.05, 0.1, 0.2], 'n_estimators': [100, 200, 400, 600, 800, 1000], 'max_depth': [-1, 5, 6, 7, 8, 9]}


In [4]:
lgb = lightgbm.LGBMClassifier()
lgb_random = RandomizedSearchCV(estimator = lgb, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [5]:
X = df_X.to_numpy()
Y = df_Y.to_numpy().squeeze()
lgb_random.fit(X, Y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END .learning_rate=0.2, max_depth=-1, n_estimators=1000; total time=   6.9s
[CV] END .learning_rate=0.2, max_depth=-1, n_estimators=1000; total time=   6.9s
[CV] END .learning_rate=0.2, max_depth=-1, n_estimators=1000; total time=   7.0s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=800; total time=   9.8s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=800; total time=   9.9s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=800; total time=  10.7s
[CV] END ..learning_rate=0.2, max_depth=5, n_estimators=1000; total time=   6.1s
[CV] END ..learning_rate=0.2, max_depth=5, n_estimators=1000; total time=   6.3s
[CV] END .learning_rate=0.05, max_depth=-1, n_estimators=800; total time=  15.9s
[CV] END ..learning_rate=0.2, max_depth=5, n_estimators=1000; total time=   6.1s
[CV] END .learning_rate=0.05, max_depth=-1, n_estimators=800; total time=  15.8s
[CV] END ...learning_rate=0.1, max_depth=8, n_

RandomizedSearchCV(cv=3, estimator=LGBMClassifier(), n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [-1, 5, 6, 7, 8, 9],
                                        'n_estimators': [100, 200, 400, 600,
                                                         800, 1000]},
                   random_state=42, verbose=2)

In [6]:
lgb_random.best_params_

{'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.1}

In [7]:
# best params
best_params = {'n_estimators': 200, 
               'max_depth': 8, 
               'learning_rate': 0.1}

Evaluate

In [8]:
skf = StratifiedKFold(n_splits=5)
f1_base = []
f1_optim = []

for train_index, test_index in skf.split(X, Y):
    clf = lightgbm.LGBMClassifier()
    
    clf.fit(X[train_index], Y[train_index])
    y_pred = clf.predict(X[test_index])
    f1_base.append(f1_score(Y[test_index], y_pred, average='micro'))
    
    clf_optim = lightgbm.LGBMClassifier(n_estimators=200, max_depth=8, learning_rate=0.1)
    clf_optim.fit(X[train_index], Y[train_index])
    y_pred = clf_optim.predict(X[test_index])
    f1_optim.append(f1_score(Y[test_index], y_pred, average='micro'))

In [9]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.8587062026515152
0.8589026606793745
